In [ ]:
YAML = """
apiVersion: "sparkoperator.hpe.com/v1beta2"
kind: SparkApplication
metadata:
  name: spark-pi-secure
spec:
  #sparkConf:
    # Note: If you are executing the application as a K8 user that MapR can verify,
    #       you do not need to specify a spark.mapr.user.secret
    #spark.mapr.user.secret: spark-user-secret
    # Note: You do not need to specify a spark.eventLog.dir
    #       it will be auto-generated with the pattern "maprfs:///apps/spark/<namespace>"
    #spark.eventLog.dir: "maprfs:///apps/spark/sampletenant"
  type: Scala
  sparkVersion: 3.1.2
  mode: cluster
  image: gcr.io/mapr-252711/spark-3.1.2:202202161825P150
  imagePullPolicy: Always
  imagePullSecrets:
  - imagepull
  mainClass: org.apache.spark.examples.SparkPi
  mainApplicationFile: "local:///opt/mapr/spark/spark-3.1.2/examples/jars/spark-examples_2.12-3.1.2.3-eep-800.jar"
  restartPolicy:
    type: Never
  driver:
    cores: 1
    coreLimit: "1000m"
    memory: "512m"
    labels:
      version: 3.1.2
    # Note: You do not need to specify a serviceAccount
    #       it will be auto-generated referencing the pre-existing "hpe-<namespace>"
    serviceAccount: hpe-testing
  executor:
    cores: 1
    coreLimit: "1000m"
    instances: 1
    memory: "512m"
    labels:
      version: 3.1.2
"""

In [ ]:
with open("pi.yaml", "w+") as f:
    f.write(YAML)

In [ ]:
from ezmllib.spark import submit, delete, logs

In [ ]:
#delete("spark-pi-secure")
submit(yaml_path="pi.yaml")

In [ ]:
%%bash
#POD=spark-pi-secure-driver
timestamp=$(date +%s --date="$giveDate 60 minutes")
while true ; do
    current_timestamp=$(date +%s)
    POD=$(kubectl get pods --no-headers -o custom-columns=":metadata.name" | grep spark-pi-secure-driver)
    if [ -n "$POD" ]; then
        STATUS=$(kubectl get pod $POD --no-headers | awk '{ print $3 }')
        if [ "$STATUS" == "Error" ]; then
            echo "Test Failed."
            break
        fi
        if [ "$STATUS" == "ImagePuErrorllBackOff" ]; then
            echo "Test Failed."
            break
        fi
        if [ "$STATUS" == "Completed" ]; then
            echo "Test Passed."
            break
        fi
    sleep 20s
    fi
    
    if [ "$current_timestamp" -gt "$timestamp" ]; then
        echo "Test Failed."
        break
    fi
done

In [ ]:
!kubectl delete -f pi.yaml